<a href="https://colab.research.google.com/github/YazzRz/TFM_Ana_Rozo_Georeferenciacion_Violencia_Machista/blob/main/Final_Revisi%C3%B3n_y_validaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 1. Instalación de dependencias necesarias
# ============================================
# Incluyen librerías para scraping, NLP y geolocalización.

!pip install requests beautifulsoup4
!pip install spacy geopy
!pip install newspaper3k
!pip install lxml_html_clean
!pip install geotext

# Modelos de spaCy para procesamiento de texto en español
!python -m spacy download es_core_news_md
!python -m spacy download es_core_news_sm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 70.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.7 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=d344f92401198c78e0081af4ebce01d0ba9ffb410d290c4ed453b2031606213d
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=80660b00e487ee34157fbd27f7f153f580c2e14b81c487c8f7f5c197f8ab090a
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

In [ ]:
# ============================================
# 2. Diccionarios y Listas de Configuración
# ============================================

# ------------------------------------------------
# Mapeo de periódicos
# ------------------------------------------------
# Este diccionario permite traducir el nombre corto del periódico
# en su información oficial: nombre completo y país.
# Sirve para enriquecer los metadatos de cada noticia procesada.

MAPA_MEDIOS = {
    "elcolombiano":     {"diario": "El Colombiano", "pais": "COLOMBIA"},
    "elpais":           {"diario": "El País", "pais": "ESPAÑA"},
    "espectador":       {"diario": "El Espectador", "pais": "COLOMBIA"},
    "eltiempo":         {"diario": "El Tiempo", "pais": "COLOMBIA"},
    "heraldo":          {"diario": "El Heraldo", "pais": "COLOMBIA"},
    "eluniversal":      {"diario": "El Universal", "pais": "COLOMBIA"},
    "abc":               {"diario": "ABC", "pais": "ESPAÑA"},
    "elmundo":           {"diario": "El Mundo", "pais": "ESPAÑA"},
    "larazon":           {"diario": "La Razón", "pais": "ESPAÑA"},
    "publico":           {"diario": "Público", "pais": "ESPAÑA"},
}
# ------------------------------------------------
# Lista de países hispanohablantes
# ------------------------------------------------
# Usada para detección de ubicaciones en los textos.
# Facilita filtrar noticias de países relevantes.

PAISES_HISPANOS = [
    "Argentina", "Bolivia", "Chile", "Colombia", "Costa Rica", "Cuba",
    "Ecuador", "El Salvador", "España", "Guatemala", "Honduras",
    "México", "Nicaragua", "Panamá", "Paraguay", "Perú", "Puerto Rico",
    "República Dominicana", "Uruguay", "Venezuela"
]

# ------------------------------------------------
# Tokens principales
# ------------------------------------------------
# Palabras clave que representan distintas formas de violencia machista.
# Se usarán para conteo simple de ocurrencias dentro de las noticias.

tokens = [
    "abuso sexual", "acoso sexual", "acoso callejero", "acoso laboral", "acoso escolar",
    "agresion sexual", "ciberviolencia", "feminicidio", "grooming", "explotacion sexual",
    "matrimonio forzado", "matrimonio infantil", "mutilacion genital", "sumision quimica",
    "trata de mujeres", "violencia machista", "machismo", "misoginia", "sexismo",
    "violacion", "violaciones", "violencia bajo el efecto de sustancias", "violencia de genero",
    "violencia digital", "violencia domestica", "violencia familiar", "violencia intrafamiliar",
    "violencia economica", "violencia patrimonial", "violencia vicaria", "violencia psicologica",
    "violencia emocional", "microagresiones", "violencia simbolica", "violencia estructural",
    "violencia institucional", "violencia docente", "violencia laboral", "violencia en la comunidad",
    "violencia fisica", "violencia sexual", "violencia sexual cibernetica",
    "violencia sobre la salud sexual y reproductiva", "discriminacion contra la mujer",
    "discriminacion femenina", "dominacion masculina", "techo de cristal",
    "perspectiva de genero", "asesinatos de mujeres", "revictimización", "impunidad",
    "hostigamiento sexual", "estupro", "pornografia", "acto sexual violento",
    "acceso carnal violento", "induccion a la prostitucion", "constreñimiento a la prostitucion",
    "crimenes sexuales", "crimenes por motivo de genero", "ciberacoso", "ciberhostigamiento"
]

# ------------------------------------------------
# Variantes de tokens
# ------------------------------------------------
# Diccionario de sinónimos y expresiones equivalentes para cada token.
# Permite detectar violencia aunque se use una forma verbal distinta
# (ej: "violó" → se cuenta como "violacion").

variantes_token = {
    "abuso sexual": ["abuso sexual", "abusó sexualmente", "abusador", "abusadores", "abusaron sexualmente"],
    "acoso sexual": ["acoso sexual", "acosó sexualmente", "acosador", "acosadores"],
    "acoso laboral": ["acoso laboral", "hostigamiento en el trabajo"],
    "acoso escolar": ["acoso escolar", "bullying", "intimidación escolar"],
    "violacion": ["violacion", "violó", "violador", "violadores"],
    "violaciones": ["violaciones", "fueron violadas", "fueron violados"],
    "feminicidio": ["feminicidio", "feminicidios"],
    "violencia sexual": ["violencia sexual", "ataque sexual", "agresión sexual", "abuso carnal"],
    "grooming": ["grooming", "engañó a menores", "acosó a menores por internet"],
    "explotacion sexual": ["explotacion sexual", "explotación de menores", "explotación de mujeres"],
    "violencia de genero": ["violencia de genero", "violencia por razones de género"],
    "machismo": ["machismo", "actitudes machistas", "comentarios machistas"],
    "misoginia": ["misoginia", "odio hacia las mujeres", "hostilidad hacia mujeres"],
    "ciberviolencia": ["ciberviolencia", "violencia en línea", "violencia digital"],
    "impunidad": ["impunidad", "falta de castigo", "quedó libre", "no fue condenado"],
    "revictimización": ["revictimización", "culpar a la víctima", "culpabilización de la víctima"]
}

# ------------------------------------------------
# Categorías de violencia y sus tokens asociados
# ------------------------------------------------
# Cada categoría contiene una lista de palabras o expresiones clave
# que sirven para clasificar la noticia en esa tipología de violencia.
# Ejemplo: si un texto contiene "acoso laboral", se suma al conteo
# de la categoría "violencia_psicologica".

categorias_violencia = {
    "violencia_fisica": [
    "agresion sexual", "violencia fisica", "violencia domestica", "violencia intrafamiliar",
    "violencia familiar",  # ← agregar este
    "acceso carnal violento", "acto sexual violento", "violacion", "violaciones",
    "asesinatos de mujeres", "asesinadas"
],
    "violencia_sexual": [
        "abuso sexual", "acoso sexual", "hostigamiento sexual", "acto de naturaleza sexual", "grooming",
        "estupro", "pornografia", "revenge porn", "explotacion sexual", "trata de mujeres",
        "esclavitud sexual", "induccion a la prostitucion", "constrenimiento a la prostitucion",
        "sumision quimica", "ciberacoso", "ciberhostigamiento", "violencia sexual", "violencia sexual cibernetica"
    ],
    "violencia_psicologica": [
        "violencia psicologica", "microagresiones", "sexismo", "misoginia", "dominacion masculina",
        "violencia emocional", "violencia vicaria", "violencia estructural", "acoso callejero", "acoso laboral"
    ],
    "violencia_economica": [
        "violencia economica", "violencia patrimonial", "servidumbre", "servidumbre por deudas"
    ],
    "violencia_simbolica": [
        "violencia simbolica", "machismo", "techo de cristal", "cultura de violacion",
        "discriminacion femenina", "discriminacion contra la mujer"
    ],
    "violencia_institucional": [
        "violencia institucional", "violencia docente", "violencia laboral"
    ],
    "violencia_digital": [
        "ciberviolencia", "doxing", "violencia digital", "violencia sexual cibernetica",
        "ciberacoso", "ciberhostigamiento"
    ],
    "violencia_reproductiva": [
        "aborto forzado", "embarazo forzado", "esterilizacion forzada", "mutilacion genital femenina",
        "mutilacion genital", "violencia sobre la salud sexual y reproductiva"
    ],
    "violencia_genero_general": [
        "violencia de genero", "violencia contra la mujer", "feminicidio", "matrimonio forzado",
        "matrimonio infantil", "perspectiva de genero", "crimenes por motivo de genero", "crimenes sexuales"
    ]
}


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import csv
from collections import defaultdict
from datetime import datetime
import unicodedata
import time
import traceback
import spacy
from geopy.geocoders import Nominatim
from collections import Counter

# ==========================
# Configuración
# ==========================
# Modelo en español
nlp = spacy.load("es_core_news_md")

# Geolocalizador
geolocator = Nominatim(user_agent="geo_locator", timeout=10)

# ==========================
# Funciones auxiliares
# ==========================
def normalizar(texto):
    texto = texto.lower()
    texto = "".join(
        c for c in unicodedata.normalize("NFD", texto)
        if unicodedata.category(c) != "Mn"
    )
    return texto

def obtener_texto_fecha_titulo(url):
    response = requests.get(url, timeout=10)
    if response.status_code != 200:
        raise Exception(f"Estado HTTP inesperado: {response.status_code}")

    soup = BeautifulSoup(response.content, "html.parser")
    parrafos = soup.find_all('p')
    texto = " ".join(p.get_text(strip=True) for p in parrafos)
    texto = re.sub(r'\s{2,}', ' ', texto)

    meta_titulo = soup.find("meta", property="og:title")
    titulo = meta_titulo["content"] if meta_titulo and meta_titulo.get("content") else (
        soup.title.string.strip() if soup.title and soup.title.string else "sin título")

    meta_description = soup.find("meta", attrs={"name": "description"})
    subtitulo = meta_description["content"] if meta_description and meta_description.get("content") else (
        soup.find("h2").get_text(strip=True) if soup.find("h2") else "sin subtítulo")

    meta_fecha = soup.find("meta", attrs={"property": "article:published_time"})
    if meta_fecha and meta_fecha.get("content"):
        match_fecha = re.search(r"\d{4}-\d{2}-\d{2}", meta_fecha["content"])
        if match_fecha:
            fecha = datetime.strptime(match_fecha.group(), "%Y-%m-%d").strftime("%d/%m/%Y")
        else:
            fecha = "sin fecha"
    else:
        fecha = "sin fecha"

    return normalizar(texto), fecha, titulo, subtitulo, texto

def contar_tokens(texto, tokens, variantes_token):
    conteo = defaultdict(int)
    for token in tokens:
        variantes = variantes_token.get(token, [token])
        for variante in variantes:
            patron = r'\b' + re.escape(normalizar(variante)) + r's?\b'
            ocurrencias = re.findall(patron, texto)
            conteo[token] += len(ocurrencias)
    return dict(conteo)

def detectar_ubicacion_spacy(texto):
    doc = nlp(texto)
    lugares_detectados = [ent.text.strip() for ent in doc.ents if ent.label_ in ["GPE", "LOC"]]

    if not lugares_detectados:
        return {"Texto_detectado": "", "País": "", "Departamento": "", "Municipio": "", "Dirección": ""}

    lugar_principal = Counter(lugares_detectados).most_common(1)[0][0]

    try:
        loc = geolocator.geocode(lugar_principal, language="es", addressdetails=True)
        time.sleep(1)
        if loc:
            direccion = loc.raw.get("address", {})
            pais = direccion.get("country", "")
            if pais in PAISES_HISPANOS:
                return {
                    "Texto_detectado": lugar_principal,
                    "País": pais,
                    "Departamento": direccion.get("state", ""),
                    "Municipio": direccion.get("city", direccion.get("town", direccion.get("village",""))),
                    "Dirección": loc.address
                }
    except:
        pass

    return {"Texto_detectado": lugar_principal, "País": "", "Departamento": "", "Municipio": "", "Dirección": ""}

def clasificar_por_categoria(conteo, categorias):
    resultado = {}
    for categoria, lista_tokens in categorias.items():
        resultado[categoria] = sum(conteo.get(token, 0) for token in lista_tokens)
    return resultado

def extraer_fecha_de_id(id_noticia):
    try:
        # Dividir por "_" y tomar la segunda parte
        partes = id_noticia.split("_")
        if len(partes) > 1:
            bloque_fecha = partes[1]  # "20200301230012"
            # Tomar solo YYYYMMDD
            fecha = bloque_fecha[:8]
            return f"{fecha[:4]}-{fecha[4:6]}-{fecha[6:8]}"
    except Exception:
        pass
    return "fecha no encontrada"


# ==========================
# Proceso principal
# ==========================
def procesar_noticia_desde_url(id_noticia, url, periodico):
    try:
        # Verifica acceso con reintentos antes de lanzar el pipeline
        _ = obtener_html(url)
        # Extraer info del periódico desde el diccionario
        medio = MAPA_MEDIOS.get(periodico.lower(), {"diario": "Desconocido", "pais": "Desconocido"})
        texto_norm, fecha_html, titulo, subtitulo, texto_crudo = obtener_texto_fecha_titulo(url)
        fecha_id = extraer_fecha_de_id(id_noticia)
        conteo = contar_tokens(texto_norm, tokens, variantes_token)
        ubicacion = detectar_ubicacion_spacy(texto_crudo)
        conteo_categorias = clasificar_por_categoria(conteo, categorias_violencia)
        fecha_publicacion = fecha_id

        if fecha_html != "sin fecha":
            try:
                fecha_html = datetime.strptime(fecha_html, "%d/%m/%Y").strftime("%Y-%m-%d")
            except:
                fecha_html = "sin fecha"

        fecha_final = fecha_html if fecha_html != "sin fecha" else fecha_id

        estructura = {
            "ID_noticia": id_noticia,
            "url": url,
            "titulo": titulo,
            "subtitulo": subtitulo,
            "contenido": texto_crudo,
            "fecha": fecha_final,
            "diario": medio["diario"],
            "país": medio["pais"],
            "tokens_detectados": conteo,
            "conteo_por_categoria": conteo_categorias,
            "ubicacion": ubicacion,
            "estado": "procesado correctamente"
        }

    except Exception as e:
        estructura = {
            "ID_noticia": id_noticia,
            "url": url,
            "error": str(e),
            "estado": "error en la obtención o procesamiento"
        }

    return estructura



# ==========================
# Prueba
# ==========================
url = "https://web.archive.org/web/20200303235413/https://elpais.com/elpais/2020/02/03/mamas_papas/1580724180_233587.html"
resultado = procesar_noticia_desde_url("CO_20200301230012_4", url,"elpais")
print(json.dumps(resultado, indent=4, ensure_ascii=False))


{
    "ID_noticia": "CO_20200301230012_4",
    "url": "https://web.archive.org/web/20200303235413/https://elpais.com/elpais/2020/02/03/mamas_papas/1580724180_233587.html",
    "error": "name 'obtener_html' is not defined",
    "estado": "error en la obtención o procesamiento"
}


In [ ]:
import csv
import json
import time
import random
from collections import Counter
import requests
from datetime import datetime
import os

# ==========================
# Lista de User-Agents para rotar
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
    "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"
]

# ==========================
# Función de request con reintentos automáticos
def obtener_html(url, max_reintentos=3, espera_inicial=30):
    espera = espera_inicial
    for intento in range(1, max_reintentos + 1):
        headers = {"User-Agent": random.choice(user_agents)}
        try:
            resp = requests.get(url, headers=headers, timeout=60)  # timeout más largo

            if resp.status_code == 200:
                return resp.text

            elif resp.status_code == 429:
                print(f"⚠ Too Many Requests (429). Reintento {intento}/{max_reintentos} en {espera} seg...")
                time.sleep(espera)
                espera *= 2

            else:
                print(f"⚠ Estado HTTP inesperado: {resp.status_code}. Reintento {intento}/{max_reintentos} en {espera} seg...")
                time.sleep(espera)
                espera *= 2

        except requests.exceptions.RequestException as e:
            print(f"⚠ Error de conexión: {e}. Reintento {intento}/{max_reintentos} en {espera} seg...")
            time.sleep(espera)
            espera *= 2

    raise Exception("Máximo de reintentos alcanzado.")

# ==========================
# Leer archivo en csv
archivo_csv = "consolidado_url_3.csv"
with open(archivo_csv, encoding="utf-8") as f:
    lector = list(csv.DictReader(f))

# Detectar URLs repetidas
contador_urls = Counter(fila["url"] for fila in lector)
urls_repetidas = [url for url, cuenta in contador_urls.items() if cuenta > 1]

# Ajusta el tamaño de bloque (5 o 10 es manejable)
tamano_bloque = 5
bloques = [lector[i:i+tamano_bloque] for i in range(0, len(lector), tamano_bloque)]

# Crear carpeta para resultados
os.makedirs("resultados_bloques", exist_ok=True)

# Procesamiento por bloques
for i, bloque in enumerate(bloques):
    print(f"\n🔁 Procesando bloque {i+1} de {len(bloques)}\n")
    resultados = []
    errores = []

    for fila in bloque:
        id_noticia = fila["ID_noticia"]
        url = fila["url"]
        periodico = fila["periodico"]
        resultado = procesar_noticia_desde_url(id_noticia, url, periodico)
        if "error" in resultado:
            errores.append(resultado)
            print(f"⚠ Error en {id_noticia}: {resultado['error']}")
        else:
            resultados.append(resultado)
            print(f"✔ Procesada: {id_noticia}")

    # Guardar parciales por bloque
    with open(f"resultados_bloques/resultados_bloque_{i+1}.json", "w", encoding="utf-8") as f:
        json.dump(resultados, f, ensure_ascii=False, indent=4)

    with open(f"resultados_bloques/errores_bloque_{i+1}.json", "w", encoding="utf-8") as f:
        json.dump(errores, f, ensure_ascii=False, indent=4)

    print(f"💾 Guardados bloque {i+1}: {len(resultados)} resultados, {len(errores)} errores")
    print("⏳ Esperando 45 segundos antes del siguiente bloque...")
    time.sleep(45)

print("\n✅ Procesamiento completo.")

# Mostrar resumen en consola (sin guardar en archivos)
print("\n--- Resumen ---")
print(f"✔ Noticias procesadas correctamente: {len(resultados)}")
print(f"⚠ Noticias con error: {len(errores)}")
print(f"🔁 URLs repetidas: {len(urls_repetidas)}")

print("\n--- Ejemplo de resultados procesados ---")
for r in resultados[:2]:
    print(json.dumps(r, indent=4, ensure_ascii=False))



🔁 Procesando bloque 1 de 63

✔ Procesada: CO_20210119232245_0
✔ Procesada: CO_20160119082949_1
⚠ Error en CO_20171209064606_0: HTTPSConnectionPool(host='web.archive.org', port=443): Read timed out. (read timeout=10)
⚠ Estado HTTP inesperado: 404. Reintento 1/3 en 30 seg...
⚠ Estado HTTP inesperado: 404. Reintento 2/3 en 60 seg...
⚠ Estado HTTP inesperado: 404. Reintento 3/3 en 120 seg...
⚠ Error en CO_20180918211224_0: Máximo de reintentos alcanzado.
✔ Procesada: CO_20200310202701_0
💾 Guardados bloque 1: 3 resultados, 2 errores
⏳ Esperando 45 segundos antes del siguiente bloque...

🔁 Procesando bloque 2 de 63

✔ Procesada: ESP_20230826090631_1
✔ Procesada: CO_20241018235343_1
✔ Procesada: CO_20231215235053_0
✔ Procesada: CO_20181228012741_0
✔ Procesada: CO_20181026153752_0
💾 Guardados bloque 2: 5 resultados, 0 errores
⏳ Esperando 45 segundos antes del siguiente bloque...

🔁 Procesando bloque 3 de 63

✔ Procesada: ESP_20231127205411_6
⚠ Estado HTTP inesperado: 404. Reintento 1/3 en 30 

In [ ]:
import os
import json
import csv

# Ruta de la carpeta donde están los resultados y errores
carpeta = "/content/resultados_bloques"

# Archivos de salida
archivo_consolidado_csv = "consolidado_resultados.csv"
archivo_consolidado_json = "consolidado_resultados.json"
archivo_errores_csv = "consolidado_errores.csv"
archivo_errores_json = "consolidado_errores.json"

def unificar_archivos(prefijo, salida_csv, salida_json):
    archivos = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.startswith(prefijo)]
    data_total = []
    archivos_fallidos = []

    for archivo in archivos:
        try:
            with open(archivo, "r", encoding="utf-8") as f:
                contenido = f.read().strip()
                if not contenido:
                    print(f"⚠️ Archivo vacío: {archivo}")
                    archivos_fallidos.append(archivo)
                    continue

                data = json.loads(contenido)
                if isinstance(data, list):
                    data_total.extend(data)
                elif isinstance(data, dict):
                    data_total.append(data)
        except Exception as e:
            print(f"⚠️ Error leyendo {archivo}: {e}")
            archivos_fallidos.append(archivo)

    if data_total:
        # Guardar JSON consolidado
        with open(salida_json, "w", encoding="utf-8") as f:
            json.dump(data_total, f, ensure_ascii=False, indent=4)

        # Guardar CSV consolidado
        with open(salida_csv, "w", encoding="utf-8", newline="") as f:
            escritor = csv.DictWriter(f, fieldnames=data_total[0].keys())
            escritor.writeheader()
            escritor.writerows(data_total)

        print(f"✅ Consolidado generado: {salida_csv} y {salida_json}")
        print(f"📊 Registros totales: {len(data_total)}")
    else:
        print(f"⚠️ No se pudo generar el consolidado para {prefijo}")

    if archivos_fallidos:
        print("\n📌 Archivos problemáticos:")
        for af in archivos_fallidos:
            print("   ", af)

# Consolidar resultados y errores
unificar_archivos("resultados_bloque_", archivo_consolidado_csv, archivo_consolidado_json)
unificar_archivos("errores_bloque_", archivo_errores_csv, archivo_errores_json)


✅ Consolidado generado: consolidado_resultados.csv y consolidado_resultados.json
📊 Registros totales: 271
✅ Consolidado generado: consolidado_errores.csv y consolidado_errores.json
📊 Registros totales: 42


In [ ]:
# Guardar resultados
with open("resultados.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, indent=4, ensure_ascii=False)

with open("errores.json", "w", encoding="utf-8") as f:
    json.dump(errores, f, indent=4, ensure_ascii=False)

NameError: name 'json' is not defined

In [ ]:
pip install fastavro

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 78.9 MB/s eta 0:00:00


In [ ]:
import json
import fastavro
import re

# Función para limpiar nombres de campos
def limpiar_nombre(nombre):
    # Reemplazar espacios y caracteres especiales por "_"
    nombre = re.sub(r'[^A-Za-z0-9_]', '_', nombre)
    # Si empieza con número, agregar prefijo
    if nombre[0].isdigit():
        nombre = "f_" + nombre
    return nombre

# Leer JSON
with open("consolidado_resultados.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Normalizar claves y convertir valores a string
def convertir_registro(registro):
    return {limpiar_nombre(k): (json.dumps(v, ensure_ascii=False) if isinstance(v, (dict, list)) else str(v))
            for k, v in registro.items()}

data_str = [convertir_registro(d) for d in data]

# Crear esquema Avro válido
schema = {
    "type": "record",
    "name": "Noticia",
    "fields": [{"name": k, "type": "string"} for k in data_str[0].keys()]
}

# Guardar en Avro
with open("consolidado_resultados.avro", "wb") as out:
    fastavro.writer(out, schema, data_str)


In [ ]:
import json
import fastavro
import re

# Función para limpiar nombres de campos
def limpiar_nombre(nombre):
    # Reemplazar espacios y caracteres especiales por "_"
    nombre = re.sub(r'[^A-Za-z0-9_]', '_', nombre)
    # Si empieza con número, agregar prefijo
    if nombre[0].isdigit():
        nombre = "f_" + nombre
    return nombre

# Leer JSON
with open("consolidado_errores.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Normalizar claves y convertir valores a string
def convertir_registro(registro):
    return {limpiar_nombre(k): (json.dumps(v, ensure_ascii=False) if isinstance(v, (dict, list)) else str(v))
            for k, v in registro.items()}

data_str = [convertir_registro(d) for d in data]

# Crear esquema Avro válido
schema = {
    "type": "record",
    "name": "Noticia",
    "fields": [{"name": k, "type": "string"} for k in data_str[0].keys()]
}

# Guardar en Avro
with open("consolidado_errores.avro", "wb") as out:
    fastavro.writer(out, schema, data_str)
